TODO
- need to duplicate into script
- need to locate the codon dictionaries from the main_package directory
- stop codon variable

In [1]:
# imports
import argparse
from Bio.SeqUtils import GC
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio import SeqIO
import math
import numpy as np
import pandas as pd
#import random
import main_package # my package

In [2]:
### SKIP until this is scripted
# parse arguments
parser = argparse.ArgumentParser()
parser.add_argument("protein", help="Name of protein sequence", type=str)
parser.add_argument("up", help="Up: 30bp 5' of sequence", type=str)
parser.add_argument("wt", help="WT: sequence of interest", type=str)
parser.add_argument("down", help="Down: 30bp 3' of sequence ", type=str)
parser.add_argument("--vect", help="Vect: sequence of interest in vector", type=str, default=False)
parser.add_argument("--codon_table", help="Specify codon table to use", type=str, default='Standard')
parser.add_argument("--homo_len", help="Length of homology arm in fwd primer", type=int, default=20)
parser.add_argument("--primer_len", help="Ideal max length of primers", type=int, default=60)
parser.add_argument("--melt_temp", help="Melting temp of fwd primer", type=int, default=50)
parser.add_argument("--rev_melt_temp", help="Melting temp of rev primer", type=int, default=55)
parser.add_argument("--syn_snp_rate", help="Percentage of synonymous SNPs 0-1", type=float, default=.05)
parser.add_argument("--stop_rate", help="Percentage of stop codon SNPs, default = keep 10% of stop SNPs", type=float, default=.10)
parser.add_argument("--rng_seed", help="Set seed for repoducibly selecting synonymous codon sites", type=int, default=42)
parser.add_argument("--out_dir", help="Local output directory e.g. 'data/'", type=str, default=.05)
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--vect VECT] [--codon_table CODON_TABLE]
                             [--homo_len HOMO_LEN] [--primer_len PRIMER_LEN]
                             [--melt_temp MELT_TEMP]
                             [--rev_melt_temp REV_MELT_TEMP]
                             [--syn_snp SYN_SNP] [--out_dir OUT_DIR]
                             protein up wt down
ipykernel_launcher.py: error: the following arguments are required: up, wt, down


SystemExit: 2

/opt/anaconda3/envs/alignparse-environment/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
# generate iupac codon dictionaries to generate doped primers
missense_dict, synonymous_dict, no_stop_dict, no_stop_syn_dict =  main_package.codon_table.iupac_codon_dicts()

# generate yeast synonymous codon dictionary (no missense variants)
yeast_synonymous_dict = main_package.codon_table.synonymous_yeast_codons_dict()

In [3]:
# .gb loop practice: MESSY
wt_input = 'k3l_test.gb'
vector_input = None

# check for vector file
wt_file = SeqIO.read(wt_input, 'genbank')
if vector_input == None:
    vector_input = wt_input
vector_file = SeqIO.read(vector_input, 'genbank')
    
wt_seq = str(wt_file.seq.upper())
vector_seq = str(vector_file.seq.upper())

In [4]:
# get start and stop of gene for codon positions
for feature in wt_file.features:
    if feature.type == 'gene':
        gene_start = feature.location.start.position
        gene_end = feature.location.end.position

1656
1923


In [5]:
# SANITY CHECKS (make an error check module?)

# check that wt and vector files are equal in length
if len(wt_seq) != len(vector_seq):
    print('Error: WildType and Vector GenBank sequences are not of equal length')
    #break
    
# check for -20 bp homology

# check that the strand is going forward

In [6]:
# hardcode test variables
homo_len = 20 ### args.homo_len
rev_melt_temp = 55 ### args.rev_melt_temp
primer_len = 60 ### args.primer_len
melt_temp = 50 ### args.melt_temp
syn_snp_rate = .05 ### args.syn_snp_rate
stop_rate = .1 ### args.stop_rate
output_prefix = 'k3l_test' # maybe have a prefix for output

In [10]:
### loop through the window

# create empty dataframe
df = pd.DataFrame()

# set RNG with seed 
rng = np.random.RandomState(42)

# this needs to be fixed (user input? yaml?)
targ_windows = ['window_1', 'window_2', 'window_3']

# setup .fa output, truncate if file exists
file = open(f"{output_prefix}.fa",'w+')

for feature in wt_file.features:
    if feature.type not in targ_windows:
        continue
    
    # START LOOP: CHANGE THIS
    win_list = []
    homo_list = []
    rev_primer_list = []
    fwd_primer_list = []
    mut_windows_list = []
    mut_name_list = []
    full_window_list = []
    full_name_list = []
    full_primer_list = []
    
    start_index = feature.location.start.position
    window_end = feature.location.end.position
    
    # loop for each sub_window
    sub_window_n = 1
    while start_index < window_end: # this could be an issue to toggle
        data_dict = {}
        data_dict['start_index'] = start_index
        data_dict['sub_window_name'] = {str(feature.type)}-{sub_window_n}
        
        # 1. homology arm
        # INPUT: data dictionary, args (homo_len)
        # OUTPUT: data dictionary
        homology_arm = vector_seq[start_index - homo_len:start_index] ### args.homo_len
        data_dict['homology_arm'] = homology_arm
        
        # 2. reverse primer
        # INPUT: data dictionary
        # OUTPUT: data dictionary
        reverse_seq = str(Seq(vector_seq[:start_index]).reverse_complement())
        reverse_primer = reverse_seq[:15]
        while mt.Tm_NN(reverse_primer) < rev_melt_temp: ### args.rev_melt_temp
            reverse_primer = reverse_seq[:len(reverse_primer)+1] ### args.rev_melt_temp
        data_dict['reverse_primer'] = reverse_primer
        
        reverse_primer_name = f'rev_{str(feature.type)}-{sub_window_n}'
        data_dict['reverse_primer_name'] = reverse_primer_name
        
        # 3. forward primer
        # INPUT: data dictionary, start index
        # OUTPUT: 
        primer_end = start_index + (primer_len - homo_len) ### args.primer_len homo_len
        if primer_end > window_end:
            primer_end == window_end
        
        primer_start = primer_end - 15
        forward_primer = vector_seq[primer_start:primer_end]

        while mt.Tm_NN(forward_primer) < melt_temp: ### args.melt_temp
            primer_start -= 1
            forward_primer = vector_seq[primer_start:primer_end]

        # check if the primer is > 28bp
        if len(forward_primer) > (primer_len - homo_len - 12): ### args.primer_len, homo_len
            # fix mut window to 12, make a long primer
            primer_start = start_index + 12
            primer_end = primer_start + 15
            forward_primer = vector_seq[primer_start:primer_end]
            while True:
                forward_primer = vector_seq[primer_start:primer_end]
                if mt.Tm_NN(forward_primer) > melt_temp and forward_primer.upper().count('G') + forward_primer.upper().count('C') > 8: ### args.melt_temp
                    break
                else:
                    primer_end += 1
        
        # even-out the primer length to accomodate codons
        else:
            # add or subtract a bp from the fwd primer to get mut_window in frame
            if (primer_start - start_index)%3 == 2:
                primer_start += 1
                forward_primer = vector_seq[primer_start:primer_end]

            elif (primer_start - start_index)%3 == 1:
                primer_start -= 1
                forward_primer = vector_seq[primer_start:primer_end]

        if primer_start > window_end:
            # make the last fwd primer
            primer_start = window_end
            primer_end = primer_start+15
            forward_primer = vector_seq[primer_start:primer_end]
            while mt.Tm_NN(forward_primer) < melt_temp:
                primer_end += 1
                forward_primer = vector_seq[primer_start:primer_end]
        data_dict['forward_primer'] = forward_primer
        
        # 4. variant window
        # INPUT: data dictionary, .tsv output, .fa output
        # OUTPUT: dataframe, .fa file lines
        mut_len = (primer_start) - start_index
        mut_end = start_index + mut_len

        def codons_list(seq):
            return [seq[i:i+3] for i in range(0, len(seq), 3)]
        
        # removing mis_list and syn_list
        wt_list = codons_list(wt_seq[start_index:mut_end])
        vect_list = codons_list(vector_seq[start_index:mut_end])

        # generate synonymous vector codon list (top 2 codons for yeast)
        synonymous_win = [yeast_synonymous_dict[i].lower() for i in vect_list]
        
        # generate iupac missense codons list (with synonymous codons)  
        doped_codons = []
        for i, wt_codon in enumerate(wt_list):            
            syn_bool = rng.choice([True, False], p=[syn_snp_rate, 1-syn_snp_rate]) ### args.syn_snp_rate
            data_dict['synonymous_codons'] = syn_bool
            
            no_stop_bool = rng.choice([True, False], p=[stop_rate, 1-stop_rate]) ### args.stop_rate
            data_dict['no_stop_codons'] = no_stop_bool
            # missense_dict, synonymous_dict, no_stop_dict, no_stop_syn_dict
            if syn_bool and no_stop_bool:
                # use no_stop_syn_dictionary
                doped_codons.append(no_stop_syn_dict[wt_codon])
            elif syn_bool and not no_stop_bool:
                # use synonymous_dictionary
                doped_codons.append(synonymous_dict[wt_codon])
            elif no_stop_bool and not syn_bool: 
                # use no_stop_dict
                doped_codons.append(no_stop_dict[wt_codon])
            else:
                # use missense dict
                doped_codons.append(missense_dict[wt_codon])
        
        # generate the mut primer and all info
        for i, iupac_list in enumerate(doped_codons):
            aa_position = int((((start_index-gene_start)/3)+1)+i)
            for iupac_codon in iupac_list:

                codon_sub = wt_list[i] + str(aa_position) + iupac_codon
                variant_win = ''.join(synonymous_win[:i] + [iupac_codon] + synonymous_win[i+1:])
                primer_name = f'{str(feature.type)}-{sub_window_n}_{codon_sub}'
                primer = homology_arm + variant_win + forward_primer
                
                # drop iupac_codon into sub_window
                sub_window = ''.join(synonymous_win[:i] + [iupac_codon] + synonymous_win[i+1:])
                full_primer = homology_arm + sub_window + forward_primer
                
                dict_keys = ['name','codon_sub','wt','position','iupac', 'sub_window', 'primer']
                dict_values = [primer_name, codon_sub, wt_list[i], aa_position, iupac_codon, sub_window, full_primer]
                for (key,value) in zip(dict_keys,dict_values):
                    data_dict[key] = value

                # append values to dataframe
                df = df.append(data_dict, ignore_index=True)
                
                # write out to .fa
                file.writelines([f">{primer_name}\n", f"{full_primer}\n"])
     
        # reset the start index for the next mini-window
        start_index = primer_start
        sub_window_n += 1  
        
file.close()

# polish dataframe
df['position'] = df['position'].astype(int)
df.drop(columns=['start_index'], inplace=True)

df['forward_primer_tm'] = df['forward_primer'].apply(lambda x: mt.Tm_NN(x)).round(1)
df['forward_primer_gc'] = df['forward_primer'].apply(GC).round(1)
df['forward_primer_len'] = df['forward_primer'].str.len()

df['reverse_primer_tm'] = df['reverse_primer'].apply(lambda x: mt.Tm_NN(x)).round(1)
df['reverse_primer_gc'] = df['reverse_primer'].apply(GC).round(1)
df['reverse_primer_len'] = df['reverse_primer'].str.len()

cols = ['name','sub_window_name','wt','position','iupac','codon_sub','synonymous_codons','no_stop_codons','primer','homology_arm','sub_window','forward_primer','forward_primer_tm','forward_primer_gc','forward_primer_len','reverse_primer','reverse_primer_name','reverse_primer_tm','reverse_primer_gc','reverse_primer_len']
df = df[cols]

# save dataframe as .tsv
df.to_csv(f'{output_prefix}.tsv', index=False, sep='\t')